In [1]:
dayObs = 'Today'
run = "None"
sensor='R10_S10'

In [2]:
from lsst.daf.butler import Butler
from astropy.table import Table
from astropy.time import Time, TimeDelta
import tabulate
import numpy

In [3]:
if dayObs=='Today':
    dayObs = (Time.now() - TimeDelta(0.5, format='jd')).iso[:10].replace('-','')

In [4]:
butler = Butler('embargo_new', collections=[
    'LSSTCam/raw/all',
    'LSSTCam/calib/unbounded',
    'LSSTCam/photodiode'
])

In [5]:
exposureList = []

if run == "None":
    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"
else:
    where=f"exposure.science_program='{run}' and instrument='LSSTCam'"    

for record in butler.registry.queryDimensionRecords("exposure", where=where):
    nimage=len(list(butler.registry.queryDatasets('raw',dataId=record.dataId)))
    hasPD=len(list(butler.registry.queryDatasets('photodiode',dataId=record.dataId)))
    exposureList.append([record.id, record, nimage, hasPD])
exposureList.sort(key=lambda x: x[0])
table = Table(rows=[ (
            f'<a href=http://lsstcam-mcm.cp.lsst.org/FITSInfo/view.html?image={record.obs_id} target="_blank">{record.obs_id}</a>',
            record.timespan.begin.iso,
            record.seq_num,
            record.observation_type,
            record.science_program,
            record.observation_reason,
            record.exposure_time,
            record.physical_filter,
            nimage,
            f'<a href=https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/CheckPDTraces?exposure={record.id} target="_blank" >PD</a>' if hasPD else "",
            f'<a href=https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/FitsHeader?expid={record.obs_id}&detector={sensor} target="_blank" >meta</a>',    
                     )
            for [id,record,nimage, hasPD] in exposureList ]
            )

In [6]:
subtable=Table(rows=[  ( record.obs_id, record.seq_num, nimage ) for [id,record,nimage, hasPD] in exposureList ], names=(  'id', 'seqnum', 'nimage'  ) )

# Possible data transfer issue

In [7]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['seqnum'])!=1], subtable[1:][numpy.diff(subtable['seqnum'])!=1]):
    print(a['id'], b['id'])

# Changes in number of images ingested

In [8]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['nimage'])!=0], subtable[1:][numpy.diff(subtable['nimage'])!=0]):
    print(a['id'], a['nimage'], b['id'], b['nimage'])

MC_C_20241014_000110 197 MC_C_20241014_000111 196
MC_C_20241014_000111 196 MC_C_20241014_000112 197
MC_C_20241014_000114 197 MC_C_20241014_000115 196
MC_C_20241014_000115 196 MC_C_20241014_000116 195
MC_C_20241014_000116 195 MC_C_20241014_000117 197
MC_C_20241014_000118 197 MC_C_20241014_000119 196
MC_C_20241014_000119 196 MC_C_20241014_000120 195
MC_C_20241014_000120 195 MC_C_20241014_000121 197
MC_C_20241014_000121 197 MC_C_20241014_000122 196
MC_C_20241014_000122 196 MC_C_20241014_000123 197
MC_C_20241014_000421 197 MC_C_20241014_000422 188
MC_C_20241014_000422 188 MC_C_20241014_000423 182
MC_C_20241014_000423 182 MC_C_20241014_000424 197
MC_C_20241014_000424 197 MC_C_20241014_000425 188
MC_C_20241014_000425 188 MC_C_20241014_000426 197
MC_C_20241014_000426 197 MC_C_20241014_000427 194
MC_C_20241014_000427 194 MC_C_20241014_000428 185
MC_C_20241014_000428 185 MC_C_20241014_000429 191
MC_C_20241014_000430 191 MC_C_20241014_000431 179
MC_C_20241014_000431 179 MC_C_20241014_000432 194


# Full records

In [10]:
print(f"The number of exposures is {len(table)}")
tabulate.tabulate(table[::-1], tablefmt='unsafehtml',headers=[
                   'obs_id', 'time', 'seqnum', 'observation_type',
                   'science_program', 'observation_reason','exposure_time',
                   'physical_filter', 'nimage', 'hasPD', 'metadata'
                  ])

The number of exposures is 723


obs_id,time,seqnum,observation_type,science_program,observation_reason,exposure_time,physical_filter,nimage,hasPD,metadata
MC_C_20241014_000723,2024-10-15 00:56:50.787012,723,engtest,unknown,engtest,15,unknown,197,,meta
MC_C_20241014_000722,2024-10-15 00:49:17.021016,722,test,unknown,unknown,15,unknown,197,,meta
MC_C_20241014_000721,2024-10-15 00:42:11.584981,721,test,unknown,unknown,15,unknown,9,,meta
MC_C_20241014_000720,2024-10-15 00:33:54.818012,720,dark,E1396,bot_persistence,15,unknown,197,,meta
MC_C_20241014_000719,2024-10-15 00:33:36.860998,719,dark,E1396,bot_persistence,15,unknown,197,,meta
MC_C_20241014_000718,2024-10-15 00:33:19.215992,718,dark,E1396,bot_persistence,15,unknown,197,,meta
MC_C_20241014_000717,2024-10-15 00:33:01.644009,717,dark,E1396,bot_persistence,15,unknown,197,,meta
MC_C_20241014_000716,2024-10-15 00:32:43.633002,716,dark,E1396,bot_persistence,15,unknown,197,,meta
MC_C_20241014_000715,2024-10-15 00:32:25.620990,715,dark,E1396,bot_persistence,15,unknown,197,,meta
MC_C_20241014_000714,2024-10-15 00:32:07.761984,714,dark,E1396,bot_persistence,15,unknown,197,,meta
